# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
import pandas as pd
from bs4 import BeautifulSoup

# Define directories

In [2]:
number = '12'
userhome = os.path.expanduser('~')
dataset = userhome + r'/mining-forum/data/forumfiles_' + number + '/'
analyze_dir = userhome + r'/mining-forum/analyze/'

# Counting number of files

In [3]:
file_list = glob.iglob(dataset + '*')
filenum = ';'.join(file_list).split(';')
filenum.sort()
print ("Found " + str(len(filenum)) + " files")

Found 43999 files


# Clean the noisy data (remove duplicate files)

In [4]:
file = []
mid_list = []
num = 0
for files in glob.iglob(dataset + "*"):    
    num += 1
    sys.stdout.write('\rExtracting file number: %i' %(num) + ' / %i' %(len(filenum)) + ' --> %s' %(files))
    sys.stdout.flush()

    f = open(files)
    f = f.read()
    
    file_num = re.search('\/(\d+)', files)
    file_num = file_num.group(1)
    
    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title

        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                for msgid in mid.find_all('a'):
                    msid.append(msgid.text)
        msid = msid[0]

        data = [file_num, msid, title]
        username = []
        uname = []
        href = []
        soup = BeautifulSoup(f, 'html.parser')
        for userlink in soup.find_all('td', attrs={'class':'MsgSpacer'}):
            for user in userlink.find_all('td', attrs={'class':'msgud'}):
                for usr in user.find_all('a', href=True):
                    href.append(usr['href'])                                        
                    uname.append(usr.text)
                    if uname[0] == '':
                        uname[0] = 'Eclipse User'
                    username = [uname[0],href[0]]
                    
                    try:
                        aut_id = re.search('(\d+)', username[1])
                        aut_id = aut_id.group()
                    except:
                        aut_id = '0'
                    
                    username.insert(len(username),aut_id)

        if msid not in mid_list:
            mid_list.append(msid)
            clean_file = [file_num, msid, title, username[0], username[1], username[2]]
            file.append(clean_file)
        else:
            pass

    except:
        pass
    
print ("\nFiltering files are finished")

Extracting file number: 43999 / 43999 --> /Users/YusufNugroho/mining-forum/data/forumfiles_12/29739
Filtering files are finished


In [5]:
file.sort()
file

[['10',
  'message #23',
  'Some thoughts',
  'Chris Gross',
  'index.php?t=usrinfo&id=8093&',
  '8093'],
 ['100',
  'message #255',
  'Webex: Target Management Meeting',
  'Doug Gaff',
  'index.php?t=usrinfo&id=13750&',
  '13750'],
 ['1000',
  'message #2645',
  'Re: GMF errors',
  'Richard Gronback',
  'index.php?t=usrinfo&id=17086&',
  '17086'],
 ['10000',
  'message #30746',
  '[BUILD] R1.0-I',
  'Alex Panchenko',
  'index.php?t=usrinfo&id=47448&',
  '47448'],
 ['10001',
  'message #30751',
  'Stumped by installation.',
  'Eclipse User',
  '/donate',
  '0'],
 ['10002',
  'message #30757',
  'Object Creation monitor Caller',
  'Eclipse User',
  '/donate',
  '0'],
 ['10003',
  'message #30761',
  'Problems with too many clicks on buttons',
  'Roberto Sanchez',
  'index.php?t=usrinfo&id=36571&',
  '36571'],
 ['10004',
  'message #30762',
  'How to get a classloader for the project in a builder',
  'Eclipse User',
  '/donate',
  '0'],
 ['10005',
  'message #30763',
  "Can't get update 

In [6]:
len(file)

43411

In [7]:
with open (analyze_dir + 'csv_files/clean_files/with_author_id/clean_files_forum_with_author_id_' + number + '.csv', 'w') as csvfile:
    writers = csv.writer(csvfile, delimiter = ",")
    header = ['file_number', 'message_id', 'title', 'author', 'author_link', 'author_id']
    writers.writerow(header)
    for item in file:
        writers.writerow(item)
print ('CSV file has been created')

CSV file has been created


In [8]:
df_thread = pd.read_csv(analyze_dir + 'csv_files/clean_files/with_author_id/clean_files_forum_with_author_id_' + number + '.csv')
df_thread

,file_number,message_id,title,author,author_link,author_id
0,10,message #23,Some thoughts,Chris Gross,index.php?t=usrinfo&id=8093&,8093
1,100,message #255,Webex: Target Management Meeting,Doug Gaff,index.php?t=usrinfo&id=13750&,13750
2,1000,message #2645,Re: GMF errors,Richard Gronback,index.php?t=usrinfo&id=17086&,17086
3,10000,message #30746,[BUILD] R1.0-I,Alex Panchenko,index.php?t=usrinfo&id=47448&,47448
4,10001,message #30751,Stumped by installation.,Eclipse User,/donate,0
5,10002,message #30757,Object Creation monitor Caller,Eclipse User,/donate,0
6,10003,message #30761,Problems with too many clicks on buttons,Roberto Sanchez,index.php?t=usrinfo&id=36571&,36571
7,10004,message #30762,How to get a classloader for the project in a ...,Eclipse User,/donate,0
8,10005,message #30763,Can't get update manager to function correctly,Eclipse User,/donate,0
9,10006,message #30770,New STEM Design Document Wiki,Daniel Ford,index.php?t=usrinfo&id=27941&,27941


# Remove threads if the author is Eclipse Webmaster

In [9]:
cols = ['file_number', 'message_id', 'title', 'author', 'author_link', 'author_id']
new_df_thread = df_thread[cols][df_thread[cols]['author_id'] != 9957]
new_df_thread

,file_number,message_id,title,author,author_link,author_id
0,10,message #23,Some thoughts,Chris Gross,index.php?t=usrinfo&id=8093&,8093
1,100,message #255,Webex: Target Management Meeting,Doug Gaff,index.php?t=usrinfo&id=13750&,13750
2,1000,message #2645,Re: GMF errors,Richard Gronback,index.php?t=usrinfo&id=17086&,17086
3,10000,message #30746,[BUILD] R1.0-I,Alex Panchenko,index.php?t=usrinfo&id=47448&,47448
4,10001,message #30751,Stumped by installation.,Eclipse User,/donate,0
5,10002,message #30757,Object Creation monitor Caller,Eclipse User,/donate,0
6,10003,message #30761,Problems with too many clicks on buttons,Roberto Sanchez,index.php?t=usrinfo&id=36571&,36571
7,10004,message #30762,How to get a classloader for the project in a ...,Eclipse User,/donate,0
8,10005,message #30763,Can't get update manager to function correctly,Eclipse User,/donate,0
9,10006,message #30770,New STEM Design Document Wiki,Daniel Ford,index.php?t=usrinfo&id=27941&,27941


In [10]:
new_df_thread.to_csv(analyze_dir + 'csv_files/clean_files/with_author_id/cleaned/threads_without_bot_authors_forum_' + number + '.csv', index=False)
print ("CSV file has been created")

CSV file has been created
